In [1]:
import scipy.io as sio
import glob
import numpy
import time

In [2]:
def load_patient_train_data(paths):
# Load training data for patient


    X = []
    Y = []

    print('...loading train data')
    start = time.time()

    for path in sorted(glob.glob(paths), key=numericalSort):
        X.append(sio.loadmat(path))
        Y.append(int(path[-5]))
    
    Y = numpy.array(Y)
    print('time elapsed: %s sec' %(time.time() - start))
    
    
    return(X, Y)

In [3]:
def load_patient_test_data(paths):
# Load training data for patient


    X = []
    file_array = []

    print('...loading test data')
    start = time.time()

    for path in sorted(glob.glob(paths), key=numericalSort):
        X.append(sio.loadmat(path))
        file_array.append(os.path.split(path)[1])
    
    print('time elapsed: %s sec' %(time.time() - start))
    
    
    return(X, file_array)

In [4]:
def get_channel1_data(X):
# extract data from channel 1    
    
    
    data_channel1 = numpy.zeros([240000, len(X)])

    for i in xrange(len(X)):
        data_channel1[:,i] = X[i]['dataStruct']['data'][0][0][:,0]
    
    
    return(data_channel1)

In [5]:
# The numericalSort function splits out any digits in a filename, 
# turns it into an actual number, and returns the result for sorting

import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

## fft computation needs to be done in batches because I do not have enought memory to hold result of fft on entire dataset

In [21]:
def n_sample_fft(X, samples, index):
    
    num_freq= 240000
    channels = 16
    X_fft = numpy.zeros([samples, num_freq, channels])

    for i in xrange(samples):
            X_fft[i,:,:] = numpy.absolute(numpy.fft.fft(X[i+index]['dataStruct']['data'][0][0], axis=0))
            
    return X_fft

In [56]:
def n_sample_half_power_freq(X_fft, samples):

    channels = 16
    power_percentage = 0.5
    half_pwr_freq = numpy.zeros([samples, channels])

    for i in xrange(samples):

        #X_sample_pwr = numpy.linalg.norm(X_fft[i,:,j:j+1], axis=1)*numpy.linalg.norm(X_fft[i,:,j:j+1], axis=1)/float(X_fft.shape[1])
        X_sample_pwr = X_fft[i,:,:]*X_fft[i,:,:]/float(X_fft.shape[1])
        X_sample_pwr_sum = numpy.sum(X_sample_pwr[0:N/2], axis=0)

        for j in xrange(channels):
            S = 0
            for k in xrange(X_sample_pwr.shape[0]):
                if S >= X_sample_pwr_sum[j]*power_percentage:
                    half_pwr_freq[i,j] = xf[k]
                    break 
                S = S + X_sample_pwr[k,j]
                
    return half_pwr_freq

In [ ]:
# load training data
X_train1, Y_train1 = load_patient_train_data('F:/Kaggle/Seizure Prediction/train_1/*.mat')

In [57]:
# compute half power frequency for each sample (1-3000) and each channel (1-16)
bins = 10
X_train1_len = 1300
channels = 16

# create list (1 to 1300)
X_train1_index = numpy.linspace(0, X_train1_len-1, X_train1_len-1)
# Separate created list into 10 bins. The size of these bins is used to perform batch computation of FFT
# and half-power frequency calulation
# Calculation is far from optimized
data_bins = numpy.histogram(X_train1_index, bins=bins)
index = 0
half_pwr_freq = numpy.zeros([X_train1_len, channels])

for i in xrange(bins):
    fft_batch = n_sample_fft(X_train1, data_bins[0][i], index)
    
    for j in xrange(channels):
        half_pwr_freq[index: index+data_bins[0][i], :] = n_sample_half_power_freq(fft_batch, data_bins[0][i])
        
    index = index + data_bins[0][i]

In [59]:
half_pwr_freq.shape

(1300L, 16L)

In [60]:
numpy.save('FEATURE train1_half_pwr_freq', half_pwr_freq)